In [3]:
import os
os.chdir('/home/atom/cvlab/thesis/')
print(os.getcwd())

/home/atom/cvlab/thesis


# Exploratory Analysis

In [4]:
from dataset.dataset import JumpPoseDataset

pose_paths = './dataset/marker_data'
info_path = './dataset/info.xlsx'
dataset = JumpPoseDataset(pose_paths, info_path, mode='3D')

In [5]:
from chainer.dataset import concat_examples

N = len(dataset)
poses, labels = zip(*dataset)
SorF, BorG, Fz, ms, skill = zip(*labels)
len(poses), len(labels), N

(40, 40, 40)

## Pose Data

In [6]:
import numpy as np
import pandas as pd

pose_lens = [len(pose) for pose in poses]
pose_df = pd.DataFrame({
    'Sequence Lengths' : pose_lens})
pose_desc = pose_df.describe()

In [10]:
xs, ys, zs = [] ,[] ,[]
for pose in poses:
    xs = np.append(xs, np.ravel(pose.T[0]))
    zs = np.append(zs, np.ravel(pose.T[1]))
    ys = np.append(ys, np.ravel(pose.T[2]))

xyz_df = pd.DataFrame({    
    'X-coordinates': xs,
    'Y-coordinates': ys,
    'Z-coordinates': zs})
xyz_desc = xyz_df.describe()
pd.concat((pose_desc, xyz_desc), axis=1)

,Sequence Lengths,X-coordinates,Y-coordinates,Z-coordinates
count,40.000000,1.277885e+06,1.277885e+06,1.277885e+06
mean,1101.625000,-2.591426e-02,2.824043e-01,6.734720e-01
std,178.294295,2.466044e-01,9.193908e-02,5.271293e-01
min,636.000000,-4.593550e-01,-4.343230e-01,7.595000e-03
25%,1056.000000,-2.471690e-01,2.314160e-01,3.275910e-01
50%,1158.000000,-1.184320e-01,2.838870e-01,4.242400e-01
75%,1213.750000,2.019270e-01,3.378360e-01,1.145362e+00
max,1359.000000,6.038880e-01,1.130581e+00,1.834448e+00


## Pose Discription for Success/Fail

In [15]:
for x in range(2):
    pose_lens, xs, ys, zs = [], [] ,[] ,[]

    for pose, label in zip(poses, labels):
        if label[0] == x:
            pose_lens.append(len(pose))

            xs = np.append(xs, np.ravel(pose.T[0]))
            zs = np.append(zs, np.ravel(pose.T[1]))
            ys = np.append(ys, np.ravel(pose.T[2]))

            pose_df = pd.DataFrame({
                'Sequence Lengths' : pose_lens})
            pose_desc = pose_df.describe()

            xyz_df = pd.DataFrame({    
                'X-coordinates': xs,
                'Y-coordinates': ys,
                'Z-coordinates': zs})
            xyz_desc = xyz_df.describe()
            
            pd.concat((pose_desc, xyz_desc), axis=1).to_latex(f'./pose_description_{x}.tex',float_format=lambda x: '%.3f' % x)

In [ ]:
pd.concat((pose_desc, xyz_desc), axis=1).to_latex('./pose_description.tex',float_format=lambda x: '%.3f' % x)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.style as style
style.use('seaborn-dark-palette')

for data in [xs, ys, zs]:
    sns.distplot(data)
plt.title('Histogram of points for each axis')
plt.legend(['X-axis', 'Y-axis', 'Z-axis'])
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.set_xlim(-0.5, 0.5)
ax.set_ylim(-0.5, 0.5)
ax.set_zlim(0.5, 2)
xs, zs, ys = poses[0][0].T
ax.scatter(xs, ys, zs)

## Labels

In [ ]:
import pandas as pd
df1 = pd.DataFrame(
    {
        'Success or Fail':SorF,
        'Bad or Good':BorG,
        'Max Force': Fz,
        'Time of Max Force': ms,
        'Absortion Skill': skill
        })
df1.describe().to_latex('./label_description.tex',float_format=lambda x: '%.3f' % x)
df1.describe()

In [ ]:
df2 = pd.DataFrame({       'Success or Fail':SorF,
        'Bad or Good':BorG})
    
for key, item in df2.items():
    print(item.value_counts(), end='\n\n')

In [ ]:
sns.set_style("darkgrid")
sns.pairplot(df1)
plt.show()